# Fully connected NN with tensorflow

First we import the necessary modules

In [22]:
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from data_handler_CMAPS import CMAPSDataHandler
#import tensorflow.contrib.layers.xavier_initializer

# Define the NN architecture

In [23]:
def ann_initialization(input_shape, output_shape):


    X = tf.placeholder(tf.float32, input_shape, name="X")
    y = tf.placeholder(tf.float32, output_shape, name="y")
    
    weights = {}
    biases = {}
    
    #Weights declaration
        
    W1 = tf.Variable(name="W1", shape=(input_shape[1], 20), initializer=tf.contrib.layers.xavier_initializer())
    b1 =  tf.Variable(name="b1", shape=(1, 20), initializer=tf.zeros_initializer())
    weights["W1"] = W1
    biases["b1"] = b1
    
    W2 = tf.Variable(name="W2", shape=(20, 20), initializer=tf.contrib.layers.xavier_initializer())
    b2 =  tf.Variable(name="b2", shape=(1, 20), initializer=tf.zeros_initializer())
    weights["W2"] = W2
    biases["b2"] = b2
    
    W3 = tf.Variable(name="W3", shape=(20, output_shape), initializer=xavier_initializer())
    b3 =  tf.Variable(name="b3", shape=(1, output_shape), initializer=tf.zeros_initializer())
    weights["W3"] = W3
    biases["b3"] = b3
    
        
    return (X, y, weights, biases)



# Function to train the neural network

In [24]:
def multilayer_perceptron(X, weights, biases):
    
    W1, b1 = weights["W1"], biases["b1"]
    W2, b2 = weights["W2"], biases["b2"]
    W2, b2 = weights["W3"], biases["b3"]
    
    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.nn.relu(Z1)
    
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2 = tf.nn.relu(Z2)
    
    out = tf.add(tf.matmul(W3, A2), b3)
    
    return out 


def train_ann(X, y, params, epochs, batch_size, display_step = 1, lamba2 = 0.0):
    
    input_shape = 2
    output_shape = 1
    
    X, y, weights, biases = ann_initialization(input_shape, output_shape)
    
    predictions = multilayer_perceptron(X, weights, biases)
    regularizers = lamba2*(tf.nn.l2_loss(weights["W1"]) + tf.nn.l2_loss(weights["W2"]))
    cost = tf.metrics.mean_squared_error(predictions, y) + regularizers
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            
            avg_cost = 0.0
            total_batch = int(X.shape[0]/batch_size)
            X_batches = np.split(X, total_batch)
            y_batches = np.split(y, total_batch)
            
            #Train with the minibatches
            for i in range(total_batch):
                batch_x, batch_y = X_batches[i], y_batches[i]
                
                _, c = sess.run([optimizer, cost], feed_dict={X:batch_x, y:batch_y})
                avg_cost += c/total_batch
                
                if epoch%display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
                    
        print("Training complete!")
        print("Epoch:Final", "cost=", "{:.9f}".format(avg_cost))
        
        return params    

# Create the Data Handler

In [21]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul=max_rul,
                                  sequence_length=window_size, sequence_stride=window_stride, 
                                  data_scaler=min_max_scaler)
dHandler_cmaps.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
dHandler_cmaps.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ..., -0.66666667  0.41085271
   0.42723005]
 [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.24031008
   0.21817178]
 [-0.31325301 -0.26095487 -0.25894666 ..., -0.33333333  0.36434109
   0.67274234]
 [-0.31325301 -0.48768258 -0.33760972 ..., -0.33333333  0.06976744
   0.2604253 ]
 [-0.30120482 -0.48506649 -0.19074949 ..., -0.33333333  0.03875969
   0.14609224]]
[[ 125.]
 [ 125.]
 [ 125.]
 [ 125.]
 [ 125.]]
Testing data (X, y)
[[-0.69879518 -0.24089819 -0.55536799 ..., -0.16666667  0.03875969
   0.27312897]
 [-0.09036145 -0.21037715 -0.37744767 ..., -0.5         0.03875969
   0.01518917]
 [-0.0060241  -0.22127752 -0.10634706 ...,  0.16666667 

# Train the ANN

In [27]:
window_size = 30
window_stride = 1
max_rul = 125
num_features = len(selected_features)
shape = num_features*window_size


#Create the model
model = ann_initialization(shape, 1)

14


TypeError: 'int' object is not subscriptable